In [1]:
import sys
import os

# Projektverzeichnis
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# Füge dem sys.path hinzu, falls noch nicht vorhanden
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
# Datenzugriff
from configs.path_manager import get_path
from models import db, Routing, Job, Schedule, JobOperation


# Utils
import src.utils.presenter as show
from src.utils.production_process import convert
from src.utils.production_process import initialize as init
from src.utils.analize import check_constrains as check

from src.utils.production_process import time_determination as term
from src.utils.production_process import gen_deadline as gen_deadline
from src.utils.production_process import filter as process_filter

# Solver Model
from src.models.cp import lateness_scheduling as cp_late_schedule
from src.models.cp import lateness_rescheduling as cp_late_reschedule
import src.models.heuristics.fcfs as fcfs


# Simulation
from src.simulation.ProductionSimulation import ProductionSimulation


# Extern
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 21)

In [3]:
max_time = 60 * 30 # 30 min

## Laden der Produktionsaufträge

In [4]:
data_path = get_path("data", "lateness")
file_template = "02_{text}.csv"

In [5]:
df_job_times_all = Job.get_dataframe(version="lateness")
df_job_times_all

,Job,Routing_ID,Arrival,Ready Time,Deadline,Status,Version
0,J25-0000,8,79,1440,3660,open,lateness
1,J25-0001,6,129,1440,2338,closed,lateness
2,J25-0002,0,198,1440,3744,open,lateness
3,J25-0003,2,243,1440,3379,open,lateness
4,J25-0004,3,287,1440,4856,open,lateness
...,...,...,...,...,...,...,...
7245,J25-7245,9,518089,518400,519640,open,lateness
7246,J25-7246,6,518116,518400,520157,open,lateness
7247,J25-7247,1,518184,518400,520172,open,lateness
7248,J25-7248,0,518188,518400,520490,open,lateness


In [6]:
def get_jssp_for_jobs(df_jobs_times, df_routings):
    """
    Erstellt ein JSSP-kompatibles DataFrame durch Verknüpfung von Job-Zeitdaten mit Routings.

    Parameter:
    - df_jobs_times: DataFrame mit mindestens den Spalten 'Job' und 'Routing_ID'.
    - df_routings: DataFrame mit Spalte 'Routing_ID' und den zugehörigen Operationsdaten.

    Rückgabe:
    - df_jssp: DataFrame mit allen für das JSSP notwendigen Informationen, inklusive 'Job' und den Operationen.
    """
    # 1. Relevante Spalten extrahieren
    df_job_ids = df_jobs_times[['Job', 'Routing_ID']].copy()

    # 2. Merge mit df_routings über Routing_ID
    df_jssp = df_job_ids.merge(df_routings, on='Routing_ID')

    return df_jssp

# Routings
df_routings = Routing.get_dataframe()
df_jssp_all = get_jssp_for_jobs(df_job_times_all, df_routings)
df_jssp_all

,Job,Routing_ID,Operation,Machine,Processing Time
0,J25-0000,8,0,M00,76
1,J25-0000,8,1,M01,69
2,J25-0000,8,2,M03,76
3,J25-0000,8,3,M05,51
4,J25-0000,8,4,M02,85
...,...,...,...,...,...
72495,J25-7249,4,5,M04,69
72496,J25-7249,4,6,M08,21
72497,J25-7249,4,7,M07,49
72498,J25-7249,4,8,M09,72


## Laden der Initialisierungsdaten vom Tag 1

In [7]:
day_numb = 1

In [8]:
df_schedule = Schedule.get_schedule_as_dataframe(date = 1, version="lateness")
df_schedule

,Job,Routing_ID,Operation,Machine,Arrival,Deadline,Start,Processing Time,End,Lateness,Tardiness,Earliness
0,J25-0005,1,0,M00,498,3106,1440,43,1483,-1623,0,1623
1,J25-0007,5,0,M02,925,2929,1440,84,1524,-1405,0,1405
2,J25-0013,3,0,M01,1192,2567,1440,81,1521,-1046,0,1046
3,J25-0011,1,0,M00,1030,3210,1483,43,1526,-1684,0,1684
4,J25-0001,6,0,M01,129,2338,1521,46,1567,-771,0,771
...,...,...,...,...,...,...,...,...,...,...,...,...
155,J25-0009,7,9,M03,1004,3578,3499,79,3578,0,0,0
156,J25-0000,8,9,M08,79,3660,3586,74,3660,0,0,0
157,J25-0002,0,9,M09,198,3744,3723,21,3744,0,0,0
158,J25-0012,2,9,M04,1103,4385,4352,33,4385,0,0,0


In [11]:
- 

SyntaxError: invalid syntax (3213666384.py, line 1)

In [9]:
df_ops_in_progress = JobOperation.get_dataframe(version="lateness", status="in progress")
df_ops_in_progress

,Job,Machine,Operation,Start,End,Processing Time,Version,Operation Status
0,J25-0004,M08,5,2846.83,2897.91,51.08,lateness,in progress
1,J25-0006,M04,8,2805.73,2886.46,80.73,lateness,in progress
2,J25-0012,M09,8,2834.21,2881.03,46.82,lateness,in progress


In [ ]:
# für die Vergangenheit geplante Operationen, die nicht geschaft abgearbeitet wurden 
file = file_template.format(text= f"plan_{day_numb:02d}_undone")
df_undone = pd.read_csv(data_path / file)
df_undone

In [10]:
#  JSSP zu allen "offenen" Jobs, mit "offenen" Operationen
unique_jobs = df_schedule.Job.unique().tolist()
df_plan_undone  = JobOperation.get_dataframe(version="lateness", jobs=unique_jobs, status="open")
df_plan_undone 

,Job,Machine,Operation,Start,End,Processing Time,Version,Operation Status
0,J25-0000,M04,8,None,None,26.0,lateness,open
1,J25-0000,M08,9,None,None,74.0,lateness,open
2,J25-0002,M09,9,None,None,21.0,lateness,open
3,J25-0003,M04,9,None,None,33.0,lateness,open
4,J25-0004,M07,6,None,None,85.0,lateness,open
5,J25-0004,M03,7,None,None,98.0,lateness,open
6,J25-0004,M09,8,None,None,22.0,lateness,open
7,J25-0004,M05,9,None,None,43.0,lateness,open
8,J25-0005,M08,9,None,None,30.0,lateness,open
9,J25-0006,M07,9,None,None,45.0,lateness,open


In [ ]:
-

## Rolling Planning ab Tag 2

In [ ]:
file_template = "03_simple_{text}.csv"

In [ ]:
first_start = 2
last_planning_start = 12

day_length = 1440

In [ ]:
for day_numb in range(first_start, last_planning_start + 1):
    day_start = day_length*day_numb 
    day_end = day_start + day_length
    print(f"Day {day_numb:02d}: [{day_start}, {day_end})")

    # ------------------------ I. Operationsvorbereitung ------------------------
    #----------------------------------------------------------------------------
    
    # ---------- a) Filterung nach der aktuellen "Ready Time" ----------
    df_jssp_curr, df_job_times_curr = process_filter.jobs_by_ready_time(df_job_times_all, df_jssp_all, ready_time = day_start, verbose=True)

    
    # ---------- b) Zusammenführung mit unerledigten Operationen -------
    df_jssp_curr = process_filter.extend_with_undone_operations(df_jssp_curr, df_undone, verbose=True)
    df_times_curr = process_filter.update_times_after_operation_changes(df_job_times_all, df_jssp_curr)

    # ------------------- II. Relevante laufende Operationen -------------------------
    df_execution_important = process_filter.get_operations_running_into_day(df_execution, day_start, verbose=True)

    # ------------------------ III. Rescheduling --------------------------------
    #----------------------------------------------------------------------------

    df_schedule = cp_late_reschedule.solve_jssp_by_tardiness_and_earliness_with_fixed_ops(df_jssp_curr, df_times_curr, 
                                                                                          df_execution_important, w_t = 5,
                                                                                          reschedule_start = day_start, 
                                                                                          msg=False, timeLimit=max_time, gapRel= 0.001)
    
    file = file_template.format(text= f"schedule_{day_numb:02d}")
    df_schedule.to_csv(data_path / file, index=False)
    
    show.plot_gantt(df_schedule, perspective="Machine", title=f"Gantt-Diagramm für Schedule ab Tag {day_numb}")
    check.all_in_one(df_schedule)

    last_ops = df_schedule.sort_values(['Job', 'Operation']).drop_duplicates('Job', keep='last')
    print(show.count_column_grouped(last_ops, "Lateness", max_val = 240, steps= 60))

    # ------------------------ IV. Simulation -----------------------------------
    simulation = ProductionSimulation(df_schedule, sigma=0.2)
    df_execution = simulation.run(start_time=day_start, end_time=day_end)
    if not df_execution.empty:
        show.plot_gantt_machines(df_execution, title=f"Gantt-Diagramm für Simulationstag {day_numb}")
    else:
        print(f"Nothing executed on day {day_numb}")

    df_undone = process_filter.get_unexecuted_operations(df_schedule, df_execution)
    print("-"*70)